# Prompt Engineering with LLMs using Gemini

**Learning Objective**

1. Learn how query the Vertex Gemini API
1. Learn how to setup the Gemini API parameters 
1. Learn prompt engineering for text generation
1. Learn prompt engineering for chat applications


The Vertex AI Gemini API lets you test, customize, and deploy instances of Google's Gemini large language models (LLM) so that you can leverage the capabilities of Gemini in your applications. The Gemini family of models supports text completion, multi-turn chat, and text embeddings generation.

This notebook will provide examples of accessing pre-trained Gemini models with the API for use cases like text classification, summarization, extraction, and chat.

### Setup

In [1]:
from IPython.display import Markdown
from vertexai.preview.generative_models import (
    Content,
    GenerationConfig,
    GenerativeModel,
    Part,
)

2024-08-05 23:23:59.508424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Text generation

The cell below implements the helper function `generate_content` to generate responses from the Gemini API. 

In [2]:
def generate(
    prompt,
    model_name="gemini-1.0-pro",
):
    model = GenerativeModel(model_name)
    responses = model.generate_content(prompt, stream=True)
    return responses

In [3]:
responses = generate(
    "What are five important things to understand about large language models?"
)
for response in responses:
    print(response.text, end="")

## Five key points about large language models:

1. **They are trained on massive amounts of text data:** This data can include books, articles, websites, and other forms of text. The more data they are trained on, the better they become at understanding and generating human language.

2. **They are not sentient or conscious:** They cannot think or feel for themselves. They simply process and respond to information based on the patterns they have learned from the data they were trained on.

3. **They can be biased:** The data they are trained on may contain biases, which can lead to biased outputs. It is important to be aware of this and to use them cautiously.

4. **They are constantly evolving:** Developers are constantly improving and updating large language models. This means that their capabilities and limitations are constantly changing.

5. **They have the potential to be incredibly beneficial:** They can be used for a variety of tasks, including writing, translation, and custom

### Text Classification 

Now that we've tested our wrapper function, let us explore prompting for classification. Text classification is a common machine learning use-case and is frequently used for tasks like spam detection, sentiment analysis, topic classification, and more. 

Both **zero-shot** and **few-shot** prompting are common with text classification use cases. Zero-shot prompting is where you do not provide examples with labels in the input, and few-shot prompting is where you provide (a few) examples in the input. 

Let us start with zero-shot classification:

In [4]:
prompt = """
Classify the following: 
text: "Zero-shot prompting is really easy with Google's Gemini API."
label: technology, politics, sports 
"""

responses = generate(prompt)
for response in responses:
    print(response.text, end="")

## Classifying the text:

Based on the provided text, the most appropriate label would be:

**Technology** 

**Explanation:**

* **Zero-shot prompting:** This term is directly related to the field of artificial intelligence and machine learning, specifically within the context of large language models (LLMs). 
* **Google's Gemini API:**  "Gemini" directly refers to a large language model developed by Google AI, further strengthening the connection to technology. APIs are also primarily associated with software and technological applications. 
* **Politics and Sports:**  There is no indication or connection to either politics or sports within the provided text. 


Here is now an example of a few-shot prompting for classification. Along with increasing the accuracy of your model, few-shot prompting gives you a certain control over the output format:

In [5]:
prompt = """
What is the topic for a given text? 
- cats 
- dogs 

Text: They always sits on my lap and purr!
The answer is: cats

Text: They love to play fetch
The answer is: dogs

Text: I throw the frisbee in the water and they swim for hours! 
The answer is: dogs 

Text: They always knock things off my counter!
The answer is:
"""

responses = generate(prompt)
for response in responses:
    print(response.text, end="")

cats 


### Text Summarization

LLMs can also be used for text summarization use cases. Text summarization produces a concise and fluent summary of a longer text document. The prompt inn the cell below simply instruct Gemini to summarize a given text:

In [6]:
prompt = """
Provide a very short summary for the following:

A transformer is a deep learning model. It is distinguished by its adoption of self-attention, 
differentially weighting the significance of each part of the input (which includes the 
recursive output) data. Like recurrent neural networks (RNNs), transformers are designed to 
process sequential input data, such as natural language, with applications towards tasks such 
as translation and text summarization. However, unlike RNNs, transformers process the 
entire input all at once. The attention mechanism provides context for any position in the 
input sequence. For example, if the input data is a natural language sentence, the transformer 
does not have to process one word at a time. This allows for more parallelization than RNNs 
and therefore reduces training times.

Summary:

"""
responses = generate(prompt)
for response in responses:
    print(response.text, end="")

##  Summary:

Transformers are deep learning models that excel at processing sequential data like natural language. Unlike RNNs, they process the entire input simultaneously, thanks to a self-attention mechanism that provides context for every part of the input sequence. This allows for faster training times and makes them suitable for tasks like translation and text summarization. 


If you need the summary to be in a certain way, as for instance a bullet point summary, you can instruct Gemini to do so:

In [7]:
prompt = """
Provide four bullet points summarizing the following:

A transformer is a deep learning model. It is distinguished by its adoption of self-attention, 
differentially weighting the significance of each part of the input (which includes the 
recursive output) data. Like recurrent neural networks (RNNs), transformers are designed to 
process sequential input data, such as natural language, with applications towards tasks such 
as translation and text summarization. However, unlike RNNs, transformers process the 
entire input all at once. The attention mechanism provides context for any position in the 
input sequence. For example, if the input data is a natural language sentence, the transformer 
does not have to process one word at a time. This allows for more parallelization than RNNs 
and therefore reduces training times.

Summary:

"""
responses = generate(prompt)
for response in responses:
    print(response.text, end="")

## Transformer Summary:

* **Deep Learning Model:** A type of neural network architecture.
* **Self-Attention Mechanism:** Differentially weights the importance of different parts of the input data, including the model's output, providing context for each position in the sequence.
* **Processes Entire Input at Once:** Unlike RNNs which process data sequentially, transformers process the entire input simultaneously, allowing for faster training through parallelization.
* **Applications:** Designed for sequential input data, particularly in natural language processing tasks like translation and text summarization. 


Dialog summarization falls under the category of text summarization. 

In [8]:
prompt = """
Generate a one-liner summary of the following chat and at the end, summarize to-do's for the service rep: 

Customer: Hi! I'm reaching out to customer service because I am having issues.

Service Rep: What seems to be the problem? 

Customer: I am trying to use the Gemini API but I keep getting an error. 

Service Rep: Can you share the error with me? 

Customer: Sure. The error says: "ResourceExhausted: 429 Quota exceeded for 
      aiplatform.googleapis.com/online_prediction_requests_per_base_model 
      with base model: text-bison"
      
Service Rep: It looks like you have exceeded the quota for usage. Please refer to 
             https://cloud.google.com/vertex-ai/docs/quotas for information about quotas
             and limits. 
             
Customer: Can you increase my quota?

Service Rep: I cannot, but let me follow up with somebody who will be able to help.

Summary:
"""

responses = generate(prompt)
for response in responses:
    print(response.text, end="")

## One-liner Summary:

Customer encountered quota error while using Gemini API, service rep advises on quota limitations and next steps.

## To-Do's for Service Rep:

- **Follow up with someone who can increase customer's quota.**
- **Share the provided link on quotas and limitations with the customer.**
- **Provide an estimated timeframe for resolution.**

### Text Extraction 
Gemini can be used to extract and structure text. Text extraction can be used for a variety of purposes. One common purpose is to convert documents into a machine-readable format. This can be useful for storing documents in a database or for processing documents with software. Another common purpose is to extract information from documents. This can be useful for finding specific information in a document or for summarizing the content of a document. 

Let us start with zero-shot extraction:

In [9]:
prompt = """
Extract the ingredients from the following recipe. 
Return the ingredients in JSON format with keys: ingredient, quantity, type.

Ingredients:
* 1 tablespoon olive oil
* 1 onion, chopped
* 2 carrots, chopped
* 2 celery stalks, chopped
* 1 teaspoon ground cumin
* 1/2 teaspoon ground coriander
* 1/4 teaspoon turmeric powder
* 1/4 teaspoon cayenne pepper (optional)
* Salt and pepper to taste
* 1 (15 ounce) can black beans, rinsed and drained
* 1 (15 ounce) can kidney beans, rinsed and drained
* 1 (14.5 ounce) can diced tomatoes, undrained
* 1 (10 ounce) can diced tomatoes with green chilies, undrained
* 4 cups vegetable broth
* 1 cup chopped fresh cilantro
"""
responses = generate(prompt)
for response in responses:
    print(response.text, end="")

```json
{
  "ingredients": [
    {
      "ingredient": "Olive oil",
      "quantity": "1 tablespoon",
      "type": "Oil"
    },
    {
      "ingredient": "Onion",
      "quantity": "1",
      "type": "Vegetable"
    },
    {
      "ingredient": "Carrot",
      "quantity": "2",
      "type": "Vegetable"
    },
    {
      "ingredient": "Celery stalks",
      "quantity": "2",
      "type": "Vegetable"
    },
    {
      "ingredient": "Ground cumin",
      "quantity": "1 teaspoon",
      "type": "Spice"
    },
    {
      "ingredient": "Ground coriander",
      "quantity": "1/2 teaspoon",
      "type": "Spice"
    },
    {
      "ingredient": "Turmeric powder",
      "quantity": "1/4 teaspoon",
      "type": "Spice"
    },
    {
      "ingredient": "Cayenne pepper",
      "quantity": "1/4 teaspoon",
      "type": "Spice",
      "optional": true
    },
    {
      "ingredient": "Salt",
      "quantity": "To taste",
      "type": "Seasoning"
    },
    {
      "ingredient": "Pepper",
     

As for classification, few-shot prompting gives you more control on the format of what is extracted: 

In [10]:
prompt = """
Extract the technical specifications from the text below in JSON format.

Text: Google Nest WiFi, network speed up to 1200Mpbs, 2.4GHz and 5GHz frequencies, WP3 protocol
JSON: {
  "product":"Google Nest WiFi",
  "speed":"1200Mpbs",
  "frequencies": ["2.4GHz", "5GHz"],
  "protocol":"WP3"
}

Text: Google Pixel 7, 5G network, 8GB RAM, Tensor G2 processor, 128GB of storage, Lemongrass
JSON:
"""

responses = generate(prompt)
for response in responses:
    print(response.text, end="")

```json
{
  "product": "Google Pixel 7",
  "network": "5G",
  "RAM": "8GB",
  "processor": "Tensor G2",
  "storage": "128GB",
  "color": "Lemongrass"
}
```

## Prompt engineering for chat

The Vertex AI Gemini API for chat is optimized for multi-turn chat. Multi-turn chat is when a model tracks the history of a chat conversation and then uses that history as the context for responses.

Gemini enables you to have freeform conversations across multiple turns. The ChatSession class simplifies the process by managing the state of the conversation, so unlike with generate_content, you do not have to store the conversation history as a list.

Let's initialize the chat:

In [11]:
model = GenerativeModel("gemini-1.0-pro")
chat = model.start_chat(history=[])
chat

The ChatSession.send_message method returns the same GenerateContentResponse type as GenerativeModel.generate_content. It also appends your message and the response to the chat history:

In [12]:
response = chat.send_message(
    "In one sentence, explain how a computer works to a young child."
)
response.text

'A computer is like a super-smart friend that helps you do things by following your instructions step-by-step! \n'

Recall that within a chat session, history is preserved. This enables the model to remember things within a given chat session for context. You can see this history in the `history` attribute of the chat session object. Notice that the history is simply a list of previous input/output pairs.

In [13]:
chat.history

[role: "user"
 parts {
   text: "In one sentence, explain how a computer works to a young child."
 },
 role: "model"
 parts {
   text: "A computer is like a super-smart friend that helps you do things by following your instructions step-by-step! \n"
 }]

In [14]:
response = chat.send_message("What question did I ask you last?")
response.text

'You asked me to explain in one sentence how a computer works to a young child. \n'

### Adding Context

While the `ChatSession` class shown earlier can handle many use cases, it does make some assumptions. If your use case doesn't fit into this chat implementation it's good to remember that `ChatSession` is just a wrapper around `GenerativeModel.generate_content`. In addition to single requests, it can handle multi-turn conversations.

The individual messages are `protos.Content objects` or compatible dictionaries, as seen in previous sections. As a dictionary, the message requires role and parts keys. The role in a conversation can either be the user, which provides the prompts, or model, which provides the responses.

Pass a list of `protos.Content` objects and it will be treated as multi-turn chat:


In [15]:
model = GenerativeModel("gemini-1.5-flash")
messages = [
    {
        "role": "user",
        "parts": ["Briefly explain how a computer works to a young child."],
    }
]
responses = model.generate_content(str(messages), stream=True)

for response in responses:
    print(response.text)

Imagine
 a computer is like a really smart toy box.  

Inside, there are
 lots of tiny parts that work together, like little helpers. One part is like
 a brain, it's called the **CPU**.  It's super fast and can follow instructions really well. 

Another part is like the toy
 box itself, it's called the **memory**. It stores all the instructions and information the computer needs.  

Then there's the **screen**,
 which is like a window to the toy box. It shows you what the computer is doing and lets you play with the toys (or games!).

When you press a key on the keyboard, it tells the CPU what to do.  
The CPU then checks the memory for instructions and tells the other parts what to do.  This makes the computer do all sorts of amazing things, like showing you pictures, playing music, or even talking to you! 




To continue the conversation, add the response and another message.

In [16]:
messages.append({"role": "model", "parts": [response.text]})

messages.append(
    {
        "role": "user",
        "parts": [
            "Okay, how about a more detailed explanation to a high school student?"
        ],
    }
)

responses = model.generate_content(str(messages), stream=True)

for response in responses:
    print(response.text)

[
{'role': 'user', 'parts': ['Briefly explain how a computer
 works to a young child.']}, {'role': 'model', 'parts
': ['Imagine a computer like a big brain! It has a special place to store all its thoughts and memories, just like your brain does. This place is
 called the **memory**. Then, it has a special part that tells it what to do, like when you decide to play a game or draw a picture.
 This part is called the **processor**.  The computer uses electricity to do all these things.  It's like magic!  Whenever you want to tell the computer what to do, you use the **keyboard** or **mouse**
 to give it instructions. And it shows you what it's doing on the **screen**.']}, {'role': 'user', 'parts': ['Okay, how about a more detailed explanation to a high school student?']}, {'
role': 'model', 'parts': ['A computer works by processing information using **binary code**, which is a system of ones and zeros. Think of it like a set of light switches, each one either on (1) or off (0). These one

### Adding chat history

You can add chat history to a chat by adding messages from role user and model alternately. System messages can be set in the first part for the first message.

In [17]:
chat2 = model.start_chat(
    history=[
        Content(
            role="user",
            parts=[
                Part.from_text(
                    """
    My name is Ned. You are my personal assistant. My favorite movies are Lord of the Rings and Hobbit.
    Who do you work for?
    """
                )
            ],
        ),
        Content(role="model", parts=[Part.from_text("I work for Ned.")]),
        Content(role="user", parts=[Part.from_text("What do I like?")]),
        Content(
            role="model", parts=[Part.from_text("Ned likes watching movies.")]
        ),
    ]
)

response = chat2.send_message("Are my favorite movies based on a book series?")
Markdown(response.text)

Yes, your favorite movies, Lord of the Rings and The Hobbit, are both based on books by J.R.R. Tolkien. 


In [18]:
response = chat2.send_message("When were these books published?")
Markdown(response.text)

The Hobbit was first published in 1937, and The Lord of the Rings was published in three volumes between 1954 and 1955. 


Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [19]:
context = """
You a Billy, a customer service chatbot for Bills Books. You only answer customer questions about Bills Books and its products.
"""

examples = [
    InputOutputTextPair(
        input_text="What is the capital of Washington State?",
        output_text="Sorry, I only answer questions about Bills Books.",
    ),
    InputOutputTextPair(
        input_text="Do you sell video games?",
        output_text="Sorry, we only sell books.",
    ),
]

chat_session = create_chat_session(context=context, examples=examples)

NameError: name 'InputOutputTextPair' is not defined

In [ ]:
response = chat_session.send_message("Where should I go on my next vacation?")
response

In [ ]:
response = chat_session.send_message("What's a good fantasy novel?")
response

Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.